# 실제 분석데이터용으로 사용할 데이터셋 만들어보기
*결측치처리 + 중복값처리 + 패딩완료한 csv 파일
*resize, 정규화 진해한 이미지파일

In [1]:
import dask.dataframe as dd
import dask
import numpy as np
from dask import delayed
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure
import cv2
import json 
import dask.bag

csv파일 불러오기

In [2]:
# df = dd.read_csv('sampled_data/*/*.csv')
df = dd.read_csv('sampled_data/*/*.csv',dtype={'외부 누적일사 평균': 'object'})

중복값 처리

In [15]:
df.compute()

,측정시각,내부 온도 1 평균,내부 온도 1 최고,내부 온도 1 최저,내부 온도 2 평균,내부 온도 2 최고,내부 온도 2 최저,내부 온도 3 평균,내부 온도 3 최고,내부 온도 3 최저,...,배지 중량 최저,양액 온도 평균,양액 온도 최고,양액 온도 최저,외부 풍향 수치,외부 풍향,외부 빗물 시간,외부 누적일사 평균,양액 급액 누적,양액 배액 누적
0,2021-07-14 10:10:00,31.7,32.0,31.3,-,-,-,-,-,-,...,-,-,-,-,-,-,-,302.6,-,-
1,2021-07-14 10:00:00,31.2,31.3,30.9,-,-,-,-,-,-,...,-,-,-,-,-,-,-,260.8,-,-
2,2021-07-14 09:50:00,31.4,31.4,31.4,-,-,-,-,-,-,...,-,-,-,-,-,-,-,196.3,-,-
3,2021-07-14 09:40:00,31.3,31.4,31.1,-,-,-,-,-,-,...,-,-,-,-,-,-,-,333.5,-,-
4,2021-07-14 09:30:00,30.7,31.0,30.5,-,-,-,-,-,-,...,-,-,-,-,-,-,-,663.5,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,2021-09-12 09:50:00,25.2,25.3,25.1,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
290,2021-09-12 09:40:00,25.1,25.3,24.9,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
291,2021-09-12 09:30:00,25.4,25.5,25.4,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
292,2021-09-12 09:20:00,25.5,25.7,25.3,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [16]:
df.drop_duplicates(subset=['측정시각'])

,측정시각,내부 온도 1 평균,내부 온도 1 최고,내부 온도 1 최저,내부 온도 2 평균,내부 온도 2 최고,내부 온도 2 최저,내부 온도 3 평균,내부 온도 3 최고,내부 온도 3 최저,내부 온도 4 평균,내부 온도 4 최고,내부 온도 4 최저,내부 습도 1 평균,내부 습도 1 최고,내부 습도 1 최저,내부 습도 2 평균,내부 습도 2 최고,내부 습도 2 최저,내부 습도 3 평균,내부 습도 3 최고,내부 습도 3 최저,내부 습도 4 평균,내부 습도 4 최고,내부 습도 4 최저,내부 이슬점 평균,내부 이슬점 최고,내부 이슬점 최저,내부 CO2 평균,내부 CO2 최고,내부 CO2 최저,외부 풍속 평균,외부 풍속 최고,외부 풍속 최저,내부 EC 1 평균,내부 EC 1 최고,내부 EC 1 최저,내부 PH 1 평균,내부 PH 1 최고,내부 PH 1 최저,배지 중량 평균,배지 중량 최고,배지 중량 최저,양액 온도 평균,양액 온도 최고,양액 온도 최저,외부 풍향 수치,외부 풍향,외부 빗물 시간,외부 누적일사 평균,양액 급액 누적,양액 배액 누적
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,float64,float64,float64,object,object,object,object,object,object,object,object,object,float64,float64,float64,object,object,object,object,object,object,object,object,object,float64,float64,float64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [17]:
df.drop_duplicates(subset=['측정시각']).compute()

,측정시각,내부 온도 1 평균,내부 온도 1 최고,내부 온도 1 최저,내부 온도 2 평균,내부 온도 2 최고,내부 온도 2 최저,내부 온도 3 평균,내부 온도 3 최고,내부 온도 3 최저,...,배지 중량 최저,양액 온도 평균,양액 온도 최고,양액 온도 최저,외부 풍향 수치,외부 풍향,외부 빗물 시간,외부 누적일사 평균,양액 급액 누적,양액 배액 누적
0,2021-07-14 10:10:00,31.7,32.0,31.3,-,-,-,-,-,-,...,-,-,-,-,-,-,-,302.6,-,-
1,2021-07-14 10:00:00,31.2,31.3,30.9,-,-,-,-,-,-,...,-,-,-,-,-,-,-,260.8,-,-
2,2021-07-14 09:50:00,31.4,31.4,31.4,-,-,-,-,-,-,...,-,-,-,-,-,-,-,196.3,-,-
3,2021-07-14 09:40:00,31.3,31.4,31.1,-,-,-,-,-,-,...,-,-,-,-,-,-,-,333.5,-,-
4,2021-07-14 09:30:00,30.7,31.0,30.5,-,-,-,-,-,-,...,-,-,-,-,-,-,-,663.5,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,2021-07-11 09:10:00,35.2,35.5,35.0,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
584,2021-07-11 09:00:00,34.4,35.0,33.8,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
586,2021-07-11 08:50:00,33.3,33.7,32.8,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
292,2021-09-12 09:20:00,25.5,25.7,25.3,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


간단하게 아래처럼 처리하면 csv간 합쳐진 상태로 중복값이 사라져서 이상하게 되는듯함, csv간 겹치는 구간이 존재할것이기 때문

In [18]:
df_drop_duplicated = df.drop_duplicates(subset=['측정시각'])

In [22]:
partitions = df_drop_duplicated.to_delayed()
shapes = [part.values.shape for part in partitions]
results = dask.compute(*shapes)
results

((2694, 52),)

In [24]:
partitions = df.to_delayed()
shapes = [part.values.shape for part in partitions]
results = dask.compute(*shapes)
results

((291, 52),
 (588, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (588, 52),
 (588, 52),
 (588, 52),
 (291, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (588, 52),
 (294, 52),
 (294, 52),
 (588, 52),
 (588, 52),
 (294, 52))

#각 partition별로 처리해보자

*각 파티션은 csv를 불러온 데이터 프레임을 의미하고 여기에 values를 적용하면 어레이값으로 출력되는것임

*판다스 데이터프레임과 동일

In [28]:
#############성공##############
partitions = df.to_delayed()
shapes = [part.drop_duplicates(subset=['측정시각']).values.shape for part in partitions]
results = dask.compute(*shapes)
results

((291, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (291, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52),
 (294, 52))

csv 데이터 만들기

In [19]:
# df = dd.read_csv('sampled_data/*/*.csv')
df = dd.read_csv('sampled_data/*/*.csv',dtype={'외부 누적일사 평균': 'object'})

csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

In [73]:
#############성공##############
@delayed
def padding(data, array_len, col_len):
    pad = np.zeros((array_len, col_len))
    length = min(array_len, len(data))
    pad[:length] = data[:length]
    return pad

partitions = df.to_delayed()
datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features].values,290,9) for part in partitions]
# results = dask.compute(*datas)
# results
datas

[Delayed('padding-aa4ab051-df2f-4efe-9888-be6ed42ef509'),
 Delayed('padding-8f74c97f-e693-4bad-b844-50a3035b423c'),
 Delayed('padding-9e144c01-55b7-4885-96de-c807fd266403'),
 Delayed('padding-8eaea867-4167-4408-8211-a2f94a1ced18'),
 Delayed('padding-6b89bcb5-c00f-452e-8933-2fc791841755'),
 Delayed('padding-dd21679d-18fa-416b-a8fe-f5d1eca39f06'),
 Delayed('padding-f2633ebc-0589-446b-ae98-a9458cb5cbe0'),
 Delayed('padding-6a0f458f-fef9-40b9-9657-df6706b8aab4'),
 Delayed('padding-a2675632-2256-4299-a728-878afd748b60'),
 Delayed('padding-d0811065-6c28-4578-a80c-80218aa1d196'),
 Delayed('padding-c2eb9bdf-1b8d-4102-af03-7ec4bd06c07f'),
 Delayed('padding-dadeb3d3-4993-4996-9165-3e28a6456448'),
 Delayed('padding-82c51ed3-b555-4f5a-bb81-1c0b13c31839'),
 Delayed('padding-75e1021d-7f95-43c3-b860-dd9c0b326f40'),
 Delayed('padding-13ae18f1-f9a2-4017-be78-a3e7ecb72247'),
 Delayed('padding-73a9d9bf-0580-47b2-89e9-f09007082337'),
 Delayed('padding-7198e65d-2804-4374-be90-5bb849571a5e'),
 Delayed('padd

# 이미지데이터셋 만들기

In [3]:
# df = dd.read_csv('sampled_data/*/*.csv')
imageset = dask_image.imread.imread('sampled_data/*/*.jpg')

In [41]:
@delayed
def img_resize(img):
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255
    return img
# partitions = imageset.to_delayed()
datas = [img_resize(part) for part in imageset]
results = dask.compute(*datas)
np.array(results).shape
# # results
# datas
# results

(21, 224, 224, 3)

In [6]:

@delayed
def padding(data, array_len, col_len):
    pad = np.zeros((array_len, col_len))
    length = min(array_len, len(data))
    pad[:length] = data[:length]
    return pad

@delayed
def img_resize(img):
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255
    return img

# 
crop = {'1':'딸기','2':'토마토','3':'파프리카','4':'오이','5':'고추','6':'시설포도'}
disease = {'1':{'a1':'딸기잿빛곰팡이병','a2':'딸기흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '2':{'a5':'토마토흰가루병','a6':'토마토잿빛곰팡이병','b2':'열과','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '3':{'a9':'파프리카흰가루병','a10':'파프리카잘록병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '4':{'a3':'오이노균병','a4':'오이흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '5':{'a7':'고추탄저병','a8':'고추흰가루병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '6':{'a11':'시설포도탄저병','a12':'시설포도노균병','b4':'일소피해','b5':'축과병'}}
risk = {'1':'초기','2':'중기','3':'말기'}

label_description = {}
for key, value in disease.items():
    label_description[f'{key}_00_0'] = f'{crop[key]}_정상'
    for disease_code in value:
        for risk_code in risk:
            label = f'{key}_{disease_code}_{risk_code}'
            label_description[label] = f'{crop[key]}_{disease[key][disease_code]}_{risk[risk_code]}'

global label_encoder
label_encoder = {key:idx for idx, key in enumerate(label_description)}
label_decoder = {val:key for key, val in label_encoder.items()}



@delayed
def label_encoding(label):
    global label_encoder
    encoded_label = label_encoder[label]
    return encoded_label

In [7]:
# @delayed
def getlable(jsonpath):
    with open(jsonpath, 'r') as f:
        json_file = json.load(f)

    crop = json_file['annotations']['crop']
    disease = json_file['annotations']['disease']
    risk = json_file['annotations']['risk']
    label = f'{crop}_{disease}_{risk}'
    return label
    
    

def get_label_list(labelpath_list):
    labelarr = np.array([])
    for ind,json_path in enumerate(labelpath_list):
        label = label_encoding(getlable(json_path))
        labelarr = np.append(labelarr,label)
    return labelarr


# jsonset = dask.bag.read_text('sampled_data/*/*.json').map(json.loads)
jsonset = dask.bag.read_text('sampled_data/*/*.json')
partitions = jsonset.to_delayed()
datas = [part for part in partitions]
# results = dask.compute(*datas)

# # results
# datas
# results
# .read_json('myfile.1.json')

In [3]:
from glob import glob

In [120]:
datas

[Delayed(('bag-from-delayed-87f42f49b78c8857e29ffebc50518cfc', 0)),
 Delayed(('bag-from-delayed-87f42f49b78c8857e29ffebc50518cfc', 1)),
 Delayed(('bag-from-delayed-87f42f49b78c8857e29ffebc50518cfc', 2)),
 Delayed(('bag-from-delayed-87f42f49b78c8857e29ffebc50518cfc', 3)),
 Delayed(('bag-from-delayed-87f42f49b78c8857e29ffebc50518cfc', 4)),
 Delayed(('bag-from-delayed-87f42f49b78c8857e29ffebc50518cfc', 5)),
 Delayed(('bag-from-delayed-87f42f49b78c8857e29ffebc50518cfc', 6)),
 Delayed(('bag-from-delayed-87f42f49b78c8857e29ffebc50518cfc', 7)),
 Delayed(('bag-from-delayed-87f42f49b78c8857e29ffebc50518cfc', 8)),
 Delayed(('bag-from-delayed-87f42f49b78c8857e29ffebc50518cfc', 9)),
 Delayed(('bag-from-delayed-87f42f49b78c8857e29ffebc50518cfc', 10)),
 Delayed(('bag-from-delayed-87f42f49b78c8857e29ffebc50518cfc', 11)),
 Delayed(('bag-from-delayed-87f42f49b78c8857e29ffebc50518cfc', 12)),
 Delayed(('bag-from-delayed-87f42f49b78c8857e29ffebc50518cfc', 13)),
 Delayed(('bag-from-delayed-87f42f49b78c8857

In [19]:
label_list = sorted(glob('sampled_data/*/*.json'))
y_train = get_label_list(label_list)
results = dask.compute(*y_train)
label = np.array(results)

In [21]:
label_list

['sampled_data/10000/10000.json',
 'sampled_data/10001/10001.json',
 'sampled_data/10002/10002.json',
 'sampled_data/10003/10003.json',
 'sampled_data/10004/10004.json',
 'sampled_data/10005/10005.json',
 'sampled_data/10006/10006.json',
 'sampled_data/10007/10007.json',
 'sampled_data/10008/10008.json',
 'sampled_data/10009/10009.json',
 'sampled_data/10010/10010.json',
 'sampled_data/10011/10011.json',
 'sampled_data/10013/10013.json',
 'sampled_data/10015/10015.json',
 'sampled_data/10017/10017.json',
 'sampled_data/10018/10018.json',
 'sampled_data/10019/10019.json',
 'sampled_data/10020/10020.json',
 'sampled_data/10021/10021.json',
 'sampled_data/10022/10022.json',
 'sampled_data/10023/10023.json']

In [16]:
# df = dd.read_csv('sampled_data/*/*.csv')
df = dd.read_csv('sampled_data/*/*.csv',dtype={'외부 누적일사 평균': 'object'})

csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

#############성공##############
@delayed
def padding(data, array_len, col_len):
    pad = np.zeros((array_len, col_len))
    length = min(array_len, len(data))
    pad[:length] = data[:length]
    return pad

partitions = df.to_delayed()
datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features].values,290,9) for part in partitions]
# results = dask.compute(*datas)
# results
datas[0].compute()

array([[31.7, 32. , 31.3, ..., 27.5, 27.9, 26.5],
       [31.2, 31.3, 30.9, ..., 26.7, 27.1, 26.5],
       [31.4, 31.4, 31.4, ..., 26.9, 26.9, 26.9],
       ...,
       [30.3, 30.4, 30.2, ..., 26.5, 26.6, 26.2],
       [30. , 30.1, 29.8, ..., 26.9, 27.2, 26.6],
       [29.5, 29.6, 29.4, ..., 26.4, 27. , 25.4]])

In [17]:
datas[1].compute()

array([[31.2, 31.4, 31. , ..., 22.1, 22.6, 21.5],
       [31. , 31.1, 30.9, ..., 22. , 22.6, 21.5],
       [30.9, 31. , 30.7, ..., 22.5, 23.1, 22.3],
       ...,
       [35.9, 36.4, 35.3, ..., 24.7, 25.8, 23.9],
       [34.8, 35.2, 34.2, ..., 24.6, 25.2, 23.4],
       [34.2, 34.3, 34.2, ..., 23.3, 24. , 22.8]])

In [18]:
datas[2].compute()

array([[28.5, 28.8, 28.4, ..., 18.2, 18.8, 17.3],
       [28.2, 28.3, 27.9, ..., 17.7, 17.9, 17.3],
       [28.2, 28.3, 28. , ..., 17.5, 17.9, 17.3],
       ...,
       [29.3, 29.5, 29.3, ..., 20.1, 20.8, 19.6],
       [29.4, 29.5, 29.1, ..., 20.9, 21.7, 20.5],
       [29.1, 29.1, 28.9, ..., 20.6, 20.7, 20.2]])

In [97]:
# jsonset['annotations']['crop']
jsonset.take(3)

JSONDecodeError: Expecting property name enclosed in double quotes: line 2 column 1 (char 2)

Traceback
---------
  File "/home/gob/.local/lib/python3.8/site-packages/dask/local.py", line 220, in execute_task
    result = _execute_task(task, data)
  File "/home/gob/.local/lib/python3.8/site-packages/dask/core.py", line 119, in _execute_task
    return func(*(_execute_task(a, cache) for a in args))
  File "/home/gob/.local/lib/python3.8/site-packages/dask/bag/core.py", line 2463, in safe_take
    r = list(take(n, b))
  File "/home/gob/.local/lib/python3.8/site-packages/dask/bag/core.py", line 2027, in __next__
    return self.f(*vals)
  File "/usr/lib/python3.8/json/__init__.py", line 357, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.8/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.8/json/decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, idx)


In [76]:
jsonset = dd.read_json('sampled_data/10000/10000.json').map(json.loads)
# partitions = jsonset.to_delayed()
# datas = [part for part in partitions]
jsonset

ValueError: Expected object or value